In [ ]:

import numpy as np
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from shapely.geometry import Point


In [ ]:

sites = pd.DataFrame({
    'Site': ['Purus Geoglyph', 'Pando Cluster', 'Mato Grosso', 'Loreto Mound', 'Guiana ADE'],
    'Lat': [-8.6, -10.5, -12.941, -1.0, 0.5],
    'Lon': [-65.0, -66.8, -55.301, -70.0, -59.0],
    'Evidence': [
        '300 m circular ditch, NDVI ring, GEDI break',
        '250 m square earthwork, moat, canopy flat',
        '180 m platform, NDVI hotspot',
        '250 m oval mound, canopy dip',
        'ADE patch, biomass spike'
    ]
})
sites.to_csv('top5_sites.csv', index=False)

gdf = gpd.GeoDataFrame(
    sites,
    geometry=gpd.points_from_xy(sites['Lon'], sites['Lat']),
    crs="EPSG:4326"
)
gdf.to_file('top5_sites.geojson', driver='GeoJSON')


In [ ]:

dem_path = 'SRTM_site1.tif'
with rasterio.open(dem_path) as src:
    dem = src.read(1)
    fig, ax = plt.subplots(figsize=(8,8))
    ax.imshow(dem, cmap='terrain')
    ax.set_title('Pseudo-hillshade / DEM')
    plt.axis('off')
    plt.savefig('hillshade_demo.png', dpi=300)
    plt.show()


In [ ]:

red_path = 'S2_B04_site1.tif'
nir_path = 'S2_B08_site1.tif'
with rasterio.open(red_path) as r, rasterio.open(nir_path) as n:
    red = r.read(1).astype(float)
    nir = n.read(1).astype(float)
    ndvi = (nir - red) / (nir + red + 1e-10)
    plt.imshow(ndvi, cmap='RdYlGn')
    plt.colorbar(label='NDVI')
    plt.title('NDVI')
    plt.axis('off')
    plt.savefig('ndvi_demo.png', dpi=300)
    plt.show()


In [ ]:

gedi_points = pd.DataFrame({
    'Lat': [-8.6, -10.5, -12.941, -1.0, 0.5],
    'Lon': [-65.0, -66.8, -55.301, -70.0, -59.0],
    'RH90': [15, 20, 12, 18, 22]
})
plt.scatter(gedi_points['Lon'], gedi_points['Lat'], c=gedi_points['RH90'], cmap='YlGn', s=100)
plt.colorbar(label='Canopy Height (RH90)')
plt.title('GEDI canopy at candidate sites')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.savefig('gedi_demo.png', dpi=300)
plt.show()


In [ ]:

print("Done. Outputs generated:")
print("- top5_sites.csv, top5_sites.geojson")
print("- hillshade_demo.png, ndvi_demo.png, gedi_demo.png")
